In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import ast

In [2]:
# Load the data
data = pd.read_csv("train.csv", low_memory=False)
data.drop_duplicates() # remove duplicates

,Characteristics.LotFeatures,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.features_reso.results,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,...,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.Heating,Structure.LivingArea,Structure.NewConstructionYN,Structure.ParkingFeatures,Structure.Rooms.RoomsTotal,Structure.YearBuilt,Tax.Zoning,UnitTypes.UnitTypeType
0,NaN,3200.0,NaN,NaN,NaN,NaN,NaN,"['Cooling.CeilingFans', 'Flooring.Carpet', 'Fl...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,0.0,commr,NaN
1,NaN,NaN,NaN,3.8,3.7,3.8,3.8,"['Appliances.GasRange', 'Appliances.Range', 'A...",NaN,4.2,...,NaN,NaN,['natural gas'],3175.0,False,['off alley'],6.0,NaN,NaN,NaN
2,NaN,18750.0,NaN,NaN,NaN,NaN,NaN,"['Flooring.Carpet', 'ParkingFeatures.Garage', ...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,1926.0,commr,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['CommunityFeatures.Lake', 'WaterfrontFeatures...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,agric,NaN
4,"['horses allowed', 'paddock', 'pond(s)']",NaN,NaN,3.6,NaN,NaN,3.6,"['Appliances.Refrigerator', 'Appliances.Microw...",NaN,4.2,...,NaN,NaN,['other'],0.0,False,"['unassigned', 'off street']",4.0,2006.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,NaN,NaN,NaN,3.8,NaN,NaN,3.8,"['Levels.One', 'ParkingFeatures.Attached', 'Pa...",NaN,4.1,...,NaN,2.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
107433,NaN,NaN,2.8,NaN,3.6,3.1,3.1,"['Appliances.Dishwasher', 'Appliances.Stainles...",3.2,NaN,...,0.0,NaN,"['natural gas', 'forced air']",0.0,False,"['assigned', 'off street']",9.0,1963.0,NaN,NaN
107434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['ExteriorFeatures.PrivateYard', 'ExteriorFeat...",NaN,NaN,...,NaN,2.5,NaN,NaN,False,['driveway'],NaN,NaN,NaN,NaN
107435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['ParkingFeatures.ParkingLot'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# DATA CLEANING OF STORIES


# Map the 'ImageData.style.stories.summary.label' column to numeric values
story_mapping = {
    '1_story': 1,
    '1.5_stories': 1.5,
    '2_stories': 2,
    '2.5_stories': 2.5,
    '3_stories_or_more': 3
}


# print(data['ImageData.style.stories.summary.label'].unique())
data['ImageData.style.stories.summary.label'] = data['ImageData.style.stories.summary.label'].map(story_mapping)
print(data['ImageData.style.stories.summary.label'].unique())

[nan 2.  1.  1.5 2.5 3. ]


In [4]:
data['Structure.YearBuilt'].unique()

array([   0.,   nan, 1926., 2006., 1967., 1990., 1976., 1940., 2007.,
       1969., 1928., 1946., 1930., 1965., 1927., 1968., 1958., 2022.,
       1895., 1952., 1948., 1978., 1899., 1987., 2023., 1985., 2003.,
       1960., 1910., 1925., 1999., 1959., 2021., 1950., 1894., 1905.,
       1964., 1935., 1896., 1920., 1880., 2009., 1923., 1901., 1988.,
       1900., 1907., 1911., 1889., 2000., 1974., 1970., 1924., 1893.,
       1961., 1914., 1954., 2004., 1909., 1939., 1903., 1955., 1912.,
       2019., 1918., 1890., 1951., 2005., 1971., 1995., 1908., 1984.,
       1932., 2008., 2001., 1887., 1997., 1962., 1885., 1874., 1931.,
       2024., 2012., 1953., 1994., 1941., 1947., 1957., 1863., 1949.,
       1917., 1956., 1986., 1898., 1992., 1929., 1973., 1972., 1884.,
       1996., 1938., 1855., 1998., 1852., 1888., 1892., 1913., 1906.,
       1915., 1975., 1989., 1963., 1919., 1856., 1943., 2011., 2013.,
       2020., 1981., 1904., 2002., 1982., 1979., 1977., 1980., 1922.,
       1875., 2018.,

In [5]:
# # replace edge case 
# data['Structure.YearBuilt'] = data['Structure.YearBuilt'].replace(190., np.nan)

# # convert 0 to nan on Structure.YearBuilt to remove 0 values
# data['Structure.YearBuilt'] = data['Structure.YearBuilt'].replace(0, np.nan)


In [6]:
#data segmentation
columnes_numeriques = data.select_dtypes(include=['number'])  # Selecciona columnes numèriques
# add ListingId to the numeric columns
columnes_numeriques = pd.concat([data['Listing.ListingId'], columnes_numeriques], axis=1)

columnes_categoriques = data.select_dtypes(exclude=['number'])  # Selecciona columnes no numèriques (categòriques)


In [7]:

# Treballar amb cada columna numèrica una a una
for columna in columnes_numeriques.columns:
    if(columna in ['Listing.ListingId', 'Listing.Price.ClosePrice']): continue
    # Calcular Q1, Q3 i IQR per a la columna actual (ignorant els NaN)
    Q1 = columnes_numeriques[columna].quantile(0.1)
    Q3 = columnes_numeriques[columna].quantile(0.9)
    IQR = Q3 - Q1
    
    # Límits per identificar outliers
    lower_bound = Q1 - 2 * IQR
    upper_bound = Q3 + 2 * IQR
    
    # Filtrar files dins dels límits, mantenint files amb NaN
    mask = (columnes_numeriques[columna].isna()) | \
        ((columnes_numeriques[columna] >= lower_bound) & (columnes_numeriques[columna] <= upper_bound))
    
    columnes_numeriques = columnes_numeriques[mask]

columnes_numeriques

,Listing.ListingId,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt
1,mrd09478582,NaN,NaN,3.8,3.7,3.8,3.8,NaN,4.2,4.0,...,2.0,1.0,1.0,NaN,NaN,NaN,NaN,3175.0,6.0,NaN
2,mrd09595738,18750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1926.0
3,mrd09604772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mrd09795957,NaN,NaN,3.6,NaN,NaN,3.6,NaN,4.2,NaN,...,1.0,0.0,2.0,NaN,NaN,NaN,NaN,0.0,4.0,2006.0
5,mrd10120778,12144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1967.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,mrd12159615,NaN,NaN,3.8,NaN,NaN,3.8,NaN,4.1,NaN,...,1.0,0.0,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
107433,mrd12168868,NaN,2.8,NaN,3.6,3.1,3.1,3.2,NaN,3.1,...,3.0,0.0,4.0,NaN,NaN,0.0,NaN,0.0,9.0,1963.0
107434,mrd12176382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,5.0,NaN,NaN,NaN,2.5,NaN,NaN,NaN
107435,mrd12183900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# canviar els NaN per la mitjana de la columna
columnes_numeriques = columnes_numeriques.apply(lambda col: col.fillna(col.mean(skipna=True)) if col.dtype in ['float64', 'int64'] else col)
columnes_numeriques

,Listing.ListingId,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt
1,mrd09478582,19504.188302,3.115019,3.800000,3.700000,3.800000,3.800000,3.312017,4.200000,4.000000,...,2.000000,1.000000,1.000000,691.445335,646.459785,0.932115,1.899309,3175.000000,6.000000,1970.222546
2,mrd09595738,18750.000000,3.115019,3.306732,3.209946,3.115745,3.180871,3.312017,3.500112,3.108284,...,1.912478,0.439366,3.064954,691.445335,646.459785,0.932115,1.899309,1719.039301,7.116705,1926.000000
3,mrd09604772,19504.188302,3.115019,3.306732,3.209946,3.115745,3.180871,3.312017,3.500112,3.108284,...,1.912478,0.439366,3.064954,691.445335,646.459785,0.932115,1.899309,1719.039301,7.116705,1970.222546
4,mrd09795957,19504.188302,3.115019,3.600000,3.209946,3.115745,3.600000,3.312017,4.200000,3.108284,...,1.000000,0.000000,2.000000,691.445335,646.459785,0.932115,1.899309,0.000000,4.000000,2006.000000
5,mrd10120778,12144.000000,3.115019,3.306732,3.209946,3.115745,3.180871,3.312017,3.500112,3.108284,...,1.912478,0.439366,3.064954,691.445335,646.459785,0.000000,1.899309,1719.039301,7.116705,1967.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,mrd12159615,19504.188302,3.115019,3.800000,3.209946,3.115745,3.800000,3.312017,4.100000,3.108284,...,1.000000,0.000000,2.000000,691.445335,646.459785,0.932115,2.000000,1719.039301,7.116705,1970.222546
107433,mrd12168868,19504.188302,2.800000,3.306732,3.600000,3.100000,3.100000,3.200000,3.500112,3.100000,...,3.000000,0.000000,4.000000,691.445335,646.459785,0.000000,1.899309,0.000000,9.000000,1963.000000
107434,mrd12176382,19504.188302,3.115019,3.306732,3.209946,3.115745,3.180871,3.312017,3.500112,3.108284,...,3.000000,1.000000,5.000000,691.445335,646.459785,0.932115,2.500000,1719.039301,7.116705,1970.222546
107435,mrd12183900,19504.188302,3.115019,3.306732,3.209946,3.115745,3.180871,3.312017,3.500112,3.108284,...,1.912478,0.439366,3.064954,691.445335,646.459785,0.932115,1.899309,1719.039301,7.116705,1970.222546


In [9]:
# normalize all numeric columns except ListingId and ClosePrice
# scaler = MinMaxScaler()
# columnes_numeriques = pd.DataFrame(scaler.fit_transform(columnes_numeriques), columns=columnes_numeriques.columns)
# columnes_numeriques

In [10]:
columnes_categoriques

,Characteristics.LotFeatures,ImageData.features_reso.results,ImageData.room_type_reso.results,ImageData.style.exterior.summary.label,Listing.Dates.CloseDate,Listing.ListingId,Location.Address.CensusBlock,Location.Address.CensusTract,Location.Address.City,Location.Address.CountyOrParish,...,Location.Area.SubdivisionName,Location.School.HighSchoolDistrict,Property.PropertyType,Structure.Basement,Structure.Cooling,Structure.Heating,Structure.NewConstructionYN,Structure.ParkingFeatures,Tax.Zoning,UnitTypes.UnitTypeType
0,NaN,"['Cooling.CeilingFans', 'Flooring.Carpet', 'Fl...","['Other', 'FrontOfStructure', 'Basement', 'Sta...",NaN,2023-12-22T00:00:00,mrd09114306,17-195-300-1004,17-195-300,morrison,whiteside,...,NaN,NaN,commercial sale,NaN,['central air'],NaN,False,NaN,commr,NaN
1,NaN,"['Appliances.GasRange', 'Appliances.Range', 'A...","['SideOfStructure', 'Other', 'Kitchen', 'Office']",NaN,2023-09-19T00:00:00,mrd09478582,17-97-865400-2007,17-97-865400,highland park,lake,...,NaN,112,residential,['partial'],['window/wall unit - 1'],['natural gas'],False,['off alley'],NaN,NaN
2,NaN,"['Flooring.Carpet', 'ParkingFeatures.Garage', ...","['Community', 'FrontOfStructure', 'Other', 'Ga...",bungalow,2023-12-04T00:00:00,mrd09595738,17-31-210900-2014,17-31-210900,chicago,cook,...,NaN,NaN,commercial sale,NaN,['central air'],NaN,False,NaN,commr,NaN
3,NaN,"['CommunityFeatures.Lake', 'WaterfrontFeatures...","['Yard', 'View']",NaN,2023-08-01T00:00:00,mrd09604772,NaN,NaN,wilmington,will,...,NaN,209u,farm,NaN,NaN,NaN,False,NaN,agric,NaN
4,"['horses allowed', 'paddock', 'pond(s)']","['Appliances.Refrigerator', 'Appliances.Microw...","['FrontOfStructure', 'BackOfStructure', 'Commu...",ranch,2023-09-20T00:00:00,mrd09795957,17-89-852403-1029,17-89-852403,maple park,kane,...,NaN,301,residential,['full'],"['partial', 'none']",['other'],False,"['unassigned', 'off street']",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,NaN,"['Levels.One', 'ParkingFeatures.Attached', 'Pa...",['FrontOfStructure'],ranch,2024-03-18T00:00:00,mrd12159615,17-203-30501-1034,17-203-30501,east peoria,tazewell,...,NaN,NaN,residential,['full'],NaN,NaN,False,NaN,NaN,NaN
107433,NaN,"['Appliances.Dishwasher', 'Appliances.Stainles...","['SideOfStructure', 'LivingRoom', 'Other', 'Ki...",NaN,2023-10-26T00:00:00,mrd12168868,17-31-411200-2000,17-31-411200,chicago,cook,...,NaN,['299'],residential,['full'],['central air'],"['natural gas', 'forced air']",False,"['assigned', 'off street']",NaN,NaN
107434,NaN,"['ExteriorFeatures.PrivateYard', 'ExteriorFeat...",['FrontOfStructure'],obstructed_view,2024-06-14T00:00:00,mrd12176382,17-31-411100-1004,17-31-411100,chicago,cook,...,NaN,NaN,residential,['partial'],NaN,NaN,False,['driveway'],NaN,NaN
107435,NaN,['ParkingFeatures.ParkingLot'],"['FrontOfStructure', 'AerialView']",NaN,2024-01-26T00:00:00,mrd12183900,17-197-880901-1025,17-197-880901,crest hill,will,...,NaN,NaN,commercial sale,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# for every column see how many unique values it has and its count
for col in columnes_categoriques.columns:
    print(columnes_categoriques[col].value_counts())
    print()

Characteristics.LotFeatures
['fenced yard']                                                                         8370
['common grounds']                                                                      3402
['corner lot']                                                                          3191
['landscaped']                                                                          2566
['cul-de-sac']                                                                          1205
                                                                                        ... 
['cul-de-sac', 'golf course lot', 'landscaped', 'wooded', 'mature trees']                  1
['corner lot', 'cul-de-sac', 'landscaped', 'wooded', 'mature trees', 'streetlights']       1
['irregular lot', 'lake front', 'water rights', 'water view', 'mature trees']              1
['cul-de-sac', 'irregular lot', 'landscaped', 'wooded', 'mature trees', 'garden']          1
['backs to open grnd', 'garden', 'partial 

INUTILS o REDUNDANTS -> BORRAR

In [12]:
# some fine tuning
# drop columns with too many unique values
# drop Listing.ListingId
columnes_categoriques = columnes_categoriques.drop('Listing.ListingId', axis=1)

# drop postalcodeplus4
columnes_categoriques = columnes_categoriques.drop('Location.Address.PostalCodePlus4', axis=1)

# drop location number Location.Address.StreetNumber
# numero independent del preu
columnes_categoriques = columnes_categoriques.drop('Location.Address.StreetNumber', axis=1)

# DROP Location.Address.StateOrProvince
columnes_categoriques = columnes_categoriques.drop('Location.Address.StateOrProvince', axis=1)

# DROP Location.Address.UnparsedAddress
columnes_categoriques = columnes_categoriques.drop('Location.Address.UnparsedAddress', axis=1)

# DROP Location.Address.StreetDirectionSuffix
# nombre d'ocurrencies no es prou gran per cada categoria
columnes_categoriques = columnes_categoriques.drop('Location.Address.StreetDirectionSuffix', axis=1)

# DROP UnitNumber
# nombre d'ocurrencies no es prou gran per cada categoria
columnes_categoriques = columnes_categoriques.drop('Location.Address.UnitNumber', axis=1)

# DROP Location.Address.StreetSuffix
# no es rellevant
columnes_categoriques = columnes_categoriques.drop('Location.Address.StreetSuffix', axis=1)

#drop Location.Address.StreetDirectionPrefix
# no es rellevant
columnes_categoriques = columnes_categoriques.drop('Location.Address.StreetDirectionPrefix', axis=1)

# DROP Location.Address.StreetName
# nombre d'ocurrencies no es prou gran per cada categoria
columnes_categoriques = columnes_categoriques.drop('Location.Address.StreetName', axis=1)

# DROP Location.Area.SubdivisionName
# no es rellevant
columnes_categoriques = columnes_categoriques.drop('Location.Area.SubdivisionName', axis=1)

# DROP Location.School.HighSchoolDistrict
# no es rellevant
columnes_categoriques = columnes_categoriques.drop('Location.School.HighSchoolDistrict', axis=1)

#DRop postalcode
# no es rellevant
columnes_categoriques = columnes_categoriques.drop('Location.Address.PostalCode', axis=1)


CHUNGAS BORRAR (TEMPORAL)

In [13]:
# DROP  
columnes_categoriques = columnes_categoriques.drop('UnitTypes.UnitTypeType', axis=1)
columnes_categoriques = columnes_categoriques.drop('Structure.ParkingFeatures', axis=1)
columnes_categoriques = columnes_categoriques.drop('Structure.Heating', axis=1)
columnes_categoriques = columnes_categoriques.drop('Structure.Cooling', axis=1)
columnes_categoriques = columnes_categoriques.drop('Structure.Basement', axis=1)
# columnes_categoriques = columnes_categoriques.drop('ImageData.room_type_reso.results', axis=1)
columnes_categoriques = columnes_categoriques.drop('ImageData.features_reso.results', axis=1)
columnes_categoriques = columnes_categoriques.drop('Characteristics.LotFeatures', axis=1)


Raras borrar (temporal)

In [14]:
# Drop all census
columnes_categoriques = columnes_categoriques.drop('Location.Address.CensusBlock', axis=1)
columnes_categoriques = columnes_categoriques.drop('Location.Address.CensusTract', axis=1)

# drop Listing.Date.CloseDate
columnes_categoriques = columnes_categoriques.drop('Listing.Dates.CloseDate', axis=1)

In [15]:
columnes_categoriques

,ImageData.room_type_reso.results,ImageData.style.exterior.summary.label,Location.Address.City,Location.Address.CountyOrParish,Property.PropertyType,Structure.NewConstructionYN,Tax.Zoning
0,"['Other', 'FrontOfStructure', 'Basement', 'Sta...",NaN,morrison,whiteside,commercial sale,False,commr
1,"['SideOfStructure', 'Other', 'Kitchen', 'Office']",NaN,highland park,lake,residential,False,NaN
2,"['Community', 'FrontOfStructure', 'Other', 'Ga...",bungalow,chicago,cook,commercial sale,False,commr
3,"['Yard', 'View']",NaN,wilmington,will,farm,False,agric
4,"['FrontOfStructure', 'BackOfStructure', 'Commu...",ranch,maple park,kane,residential,False,NaN
...,...,...,...,...,...,...,...
107432,['FrontOfStructure'],ranch,east peoria,tazewell,residential,False,NaN
107433,"['SideOfStructure', 'LivingRoom', 'Other', 'Ki...",NaN,chicago,cook,residential,False,NaN
107434,['FrontOfStructure'],obstructed_view,chicago,cook,residential,False,NaN
107435,"['FrontOfStructure', 'AerialView']",NaN,crest hill,will,commercial sale,NaN,NaN


In [16]:
# for every column see how many unique values it has and its count
for col in columnes_categoriques.columns:
    print(columnes_categoriques[col].value_counts())
    print()

ImageData.room_type_reso.results
['FrontOfStructure']                                                                                                                                                                                                                      6530
['Other']                                                                                                                                                                                                                                  291
['SideOfStructure']                                                                                                                                                                                                                        186
['FrontOfStructure', 'Other']                                                                                                                                                                                                              166
['BackOfStr

Chungas start

In [17]:
columnes_categoriques

,ImageData.room_type_reso.results,ImageData.style.exterior.summary.label,Location.Address.City,Location.Address.CountyOrParish,Property.PropertyType,Structure.NewConstructionYN,Tax.Zoning
0,"['Other', 'FrontOfStructure', 'Basement', 'Sta...",NaN,morrison,whiteside,commercial sale,False,commr
1,"['SideOfStructure', 'Other', 'Kitchen', 'Office']",NaN,highland park,lake,residential,False,NaN
2,"['Community', 'FrontOfStructure', 'Other', 'Ga...",bungalow,chicago,cook,commercial sale,False,commr
3,"['Yard', 'View']",NaN,wilmington,will,farm,False,agric
4,"['FrontOfStructure', 'BackOfStructure', 'Commu...",ranch,maple park,kane,residential,False,NaN
...,...,...,...,...,...,...,...
107432,['FrontOfStructure'],ranch,east peoria,tazewell,residential,False,NaN
107433,"['SideOfStructure', 'LivingRoom', 'Other', 'Ki...",NaN,chicago,cook,residential,False,NaN
107434,['FrontOfStructure'],obstructed_view,chicago,cook,residential,False,NaN
107435,"['FrontOfStructure', 'AerialView']",NaN,crest hill,will,commercial sale,NaN,NaN


In [18]:
def safe_eval(val):
    if pd.isna(val):  # Check for NaN
        return val
    try:
        return ast.literal_eval(val)
    except ValueError:
        return val

# Assuming 'data' is the DataFrame and 'ImageData.features_reso.results' is the column with array values
# columnes_categoriques['ImageData.room_type_reso.results'] = columnes_categoriques['ImageData.room_type_reso.results'].map(safe_eval)


# data_exploded = columnes_categoriques['ImageData.room_type_reso.results'].explode('ImageData.room_type_reso.results')


# # get dummies for the exploded column
# data_exploded = pd.get_dummies(data_exploded, prefix='ImageData.room_type_reso.results', prefix_sep='=')
# # condense the dummies by summing them up or bit by bit
# data_exploded = data_exploded.groupby(data_exploded.index).sum()

# # remove duplicated columns that are the same
# data_exploded = data_exploded.loc[:, ~data_exploded.columns.duplicated()]

# data_exploded

In [ ]:
# Carregar el CSV

data_exploded = {}
# Convertir les cadenes a llistes
data_exploded['ImageData.room_type_reso.results'] = columnes_categoriques['ImageData.room_type_reso.results'].apply(safe_eval)

# Trobar tots els atributs únics
atributs_unics = set()
for llista in data_exploded['ImageData.room_type_reso.results']:
    atributs_unics.update(llista)

# Crear una columna per cada atribut únic
for atribut in atributs_unics:
    data_exploded[atribut] = data_exploded['ImageData.room_type_reso.results'].apply(lambda x: atribut in x)

# Eliminar la columna original 'ImageData.room_type_reso.results'
data_exploded = data_exploded.drop(columns=['ImageData.room_type_reso.results'])

# Mostrar el resultat
print(data_exploded)

# Carregar el CSV
data_exploded = {}
# Convertir les cadenes a llistes
data_exploded['ImageData.room_type_reso.results'] = columnes_categoriques['ImageData.room_type_reso.results'].apply(safe_eval)

# Trobar tots els atributs únics
atributs_unics = set()
for llista in data_exploded['ImageData.room_type_reso.results']:
    atributs_unics.update(llista)

# Crear una columna per cada atribut únic
for atribut in atributs_unics:
    data_exploded[atribut] = data_exploded['ImageData.room_type_reso.results'].apply(lambda x: atribut in x)

# Eliminar la columna original 'ImageData.room_type_reso.results'
data_exploded = data_exploded.drop(columns=['ImageData.room_type_reso.results'])

# Mostrar el resultat
print(data_exploded)


TypeError: 'float' object is not iterable

Treure les fàcils que tenen poques ocurrències

In [ ]:
# Treballar amb cada columna categòrica una a una
# Iterar sobre les columnes categòriques
# Per cada columna, eliminar les categories que apareixen menys d'un 1% de les vegades

for column in columnes_categoriques.columns:
    value_counts = columnes_categoriques[column].value_counts()
    threshold = value_counts.max() * 0.05
    to_replace = value_counts[value_counts < threshold].index
    columnes_categoriques[column] = columnes_categoriques[column].replace(to_replace, 'Others')

columnes_categoriques


In [ ]:
# make dummies 
columnes_categoriques = pd.get_dummies(columnes_categoriques, drop_first=True)
columnes_categoriques

In [ ]:
# for every column see how many unique values it has and its count
for col in columnes_categoriques.columns:
    print(columnes_categoriques[col].value_counts())
    print()

In [ ]:
# add ListingId to the categorical columns
columnes_categoriques = pd.concat([data['Listing.ListingId'], columnes_categoriques], axis=1)
columnes_categoriques

In [ ]:
# natural join of the two dataframes
data_final = pd.merge(columnes_numeriques, columnes_categoriques, on='Listing.ListingId')

In [ ]:
data_final

In [ ]:
data_final.to_csv('columnes_categoriques_encoded.csv', index=False)

In [ ]:
columnes_numeriques.to_csv('columnes_numeriques.csv', index=False)
columnes_categoriques.to_csv('columnes_categoriques.csv', index=False)